In [2]:
import cv2 
import pytesseract

In [4]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files (x86)\Tesseract-OCR\tesseract.exe'

In [6]:
img1 = cv2.imread("D:\AmazonMLChallenge.py\preprocessed_images (2)\preprocessed_images\preprocessed_514pScQdlCL.jpg")

In [8]:
data1 = pytesseract.image_to_data(img1)

In [10]:
data1

'level\tpage_num\tblock_num\tpar_num\tline_num\tword_num\tleft\ttop\twidth\theight\tconf\ttext\n1\t1\t0\t0\t0\t0\t0\t0\t500\t500\t-1\t\n2\t1\t1\t0\t0\t0\t111\t0\t389\t92\t-1\t\n3\t1\t1\t1\t0\t0\t111\t0\t389\t92\t-1\t\n4\t1\t1\t1\t1\t0\t111\t0\t389\t92\t-1\t\n5\t1\t1\t1\t1\t1\t111\t0\t389\t92\t95.000000\t   \n5\t1\t1\t1\t1\t2\t427\t64\t73\t25\t95.000000\t \n2\t1\t2\t0\t0\t0\t176\t70\t251\t31\t-1\t\n3\t1\t2\t1\t0\t0\t176\t70\t251\t31\t-1\t\n4\t1\t2\t1\t1\t0\t176\t70\t251\t13\t-1\t\n5\t1\t2\t1\t1\t1\t176\t70\t47\t13\t96.158279\tprevent\n5\t1\t2\t1\t1\t2\t227\t70\t55\t10\t96.569710\telectrical\n5\t1\t2\t1\t1\t3\t287\t70\t57\t12\t88.293442\toverload,\n5\t1\t2\t1\t1\t4\t350\t70\t56\t10\t96.318901\tuniversal\n5\t1\t2\t1\t1\t5\t410\t70\t17\t10\t96.760269\tfor\n4\t1\t2\t1\t2\t0\t179\t85\t143\t16\t-1\t\n5\t1\t2\t1\t2\t1\t179\t85\t24\t10\t59.762299\t412V\n5\t1\t2\t1\t2\t2\t207\t87\t38\t12\t89.524155\tpower\n5\t1\t2\t1\t2\t3\t249\t87\t42\t14\t94.316422\tsource\n5\t1\t2\t1\t2\t4\t296\t87\t26\t9\t94

In [12]:
import pandas as pd
import re

# Create a pandas DataFrame from the data
from io import StringIO
df = pd.read_csv(StringIO(data1), sep='\t')

# Filter the rows where the confidence score (conf) is >= 0
valid_df = df[df['conf'] >= 0]

# Extract words and numbers from the text column
words_and_numbers = valid_df['text'].apply(lambda x: re.findall(r'\w+', x)).explode().dropna().tolist()

# Print the valid words and numbers
print(words_and_numbers)

['prevent', 'electrical', 'overload', 'universal', 'for', '412V', 'power', 'source', 'cars']


In [18]:
import re

def is_numeric(s):
    """Check if the string contains numeric characters."""
    return re.search(r'\d+', s) is not None

def find_nearest_numeric_in_list(word_list, search_key):
    """Find the nearest numeric value in the list based on the position of the search key."""
    # Find the position of the search key
    search_key_position = None
    for i, word in enumerate(word_list):
        if search_key in word:
            search_key_position = i
            break

    if search_key_position is None:
        return "Search key not found in the list."

    nearest_value = None
    min_distance = float('inf')

    # Search left of the search key
    for i in range(search_key_position - 1, -1, -1):
        word = word_list[i]
        if is_numeric(word):
            num_value = word
            distance = search_key_position - i  # Distance between the indices
            if distance < min_distance:
                min_distance = distance
                nearest_value = num_value

    return nearest_value

# Define search key
search_key = 'V'

# Find the nearest numeric value
nearest_value = find_nearest_numeric_in_list(words_and_numbers, search_key)
print(f'The nearest numeric value to the word containing "{search_key}" is {nearest_value}.')

The nearest numeric value to the word containing "V" is None.


In [20]:
import re

def preprocess_text(text):
    """Clean and format text by removing extra spaces."""
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
    return text.strip()

def parse_text_to_grid(text):
    """Convert text into a 2D grid by splitting lines and rows."""
    rows = text.split('\n')
    grid = [re.split(r'\s+', row) for row in rows]
    return grid

def is_numeric(s):
    """Check if the string is numeric, considering possible floats."""
    return s.replace('.', '', 1).isdigit()

def find_nearest_numeric(grid, search_key):
    """Find the nearest numeric value to the left or above the search key."""
    # Find the position of the search key in the grid
    search_key_position = None
    for i, row in enumerate(grid):
        if search_key in row:
            search_key_position = (i, row.index(search_key))
            break

    if not search_key_position:
        return "Search key not found in the grid."

    row_index, col_index = search_key_position
    nearest_value = None
    min_distance = float('inf')

    # Check cells to the left in the same row
    for col in range(col_index - 1, -1, -1):
        value = grid[row_index][col]
        if is_numeric(value):
            num_value = float(value)
            distance = col_index - col  # Horizontal distance
            if distance < min_distance:
                min_distance = distance
                nearest_value = value

    # Check cells above in the same column
    for row in range(row_index - 1, -1, -1):
        value = grid[row][col_index]
        if is_numeric(value):
            num_value = float(value)
            distance = row_index - row  # Vertical distance
            if distance < min_distance:
                min_distance = distance
                nearest_value = value

    return nearest_value if nearest_value is not None else "No numeric value found."

# Preprocess and parse the text
cleaned_text = preprocess_text(words_and_numbers)
grid = parse_text_to_grid(cleaned_text)

# Define search key
search_key = 'watt'

# Find the nearest numeric value to the left or above the search key
nearest_value = find_nearest_numeric(grid, search_key)
print(f'The nearest numeric value to '{search_key}' is {nearest_value}.")

SyntaxError: unterminated string literal (detected at line 65) (1805739947.py, line 65)